In [ ]:
import sys
!{sys.executable} -m pip install langchain huggingface_hub

!{sys.executable} -m  pip install bitsandbytes>=0.39.0
!{sys.executable} -m pip install --upgrade accelerate
!{sys.executable} -m pip install --upgrade sentence_transformers
!{sys.executable} -m  pip install --upgrade transformers
!{sys.executable} -m pip install --upgrade trl
!{sys.executable} -m pip install --upgrade peft
!{sys.executable} -m pip install --upgrade sqlite3
!{sys.executable} -m pip install --upgrade pytz
!{sys.executable} -m pip install --upgrade pypdf PyPDF2
!{sys.executable} -m pip install --upgrade langchain_experimental
!{sys.executable} -m pip install --upgrade langchain_community
!{sys.executable} -m pip install --upgrade langchain_openai
!{sys.executable} -m pip install --upgrade openai tiktoken chromadb
!pip install unstructured['pdf'] unstructured
!{sys.executable} -m pip install --upgrade textwrap
!pip install --upgrade --quiet  cohere
!pip install langchain-cohere
!pip install grandalf
!pip install wikipedia
!pip install faiss-cpu
!pip install faiss-gpu
!pip install langchainhub
!pip install -qU langchain-mistralai
!pip install "unstructured[html]"
!pip install bs4
!pip install rank_bm25
!pip install gradio

import gradio as gr
from huggingface_hub import InferenceClient

import os
import time
import sqlite3
import re
from pytz import timezone
import pytz
import datetime
import json
import textwrap
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
from operator import itemgetter
import os
import sys

import peft
import bitsandbytes
import accelerate
import trl
from accelerate import init_empty_weights
import torch
import pandas as pd

from transformers import AutoConfig, BitsAndBytesConfig
from accelerate.utils import BnbQuantizationConfig, load_and_quantize_model
from accelerate import load_checkpoint_and_dispatch
import accelerate, bitsandbytes

from langchain_mistralai import ChatMistralAI
import getpass
from google.colab import userdata


from langchain import hub
from langchain import PromptTemplate, HuggingFaceHub, LLMChain
from langchain_experimental.agents import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import UnstructuredPDFLoader, PyMuPDFLoader
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_community.document_loaders import BSHTMLLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import WebBaseLoader
from langchain.vectorstores import Chroma

from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

from langchain.chains import RetrievalQA
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda
from langchain.llms import HuggingFaceHub
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.callbacks.tracers import ConsoleCallbackHandler

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import AgentType, initialize_agent
from langchain_core.messages import HumanMessage, ToolMessage, SystemMessage, AIMessage
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.tools import Tool
from langchain_experimental.utilities import PythonREPL
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from google.colab import userdata


Instantiate your llm - using Mistral in this example

In [ ]:
Mistral_TOKEN = userdata.get('mistralapi')
os.environ['MISTRAL_API_KEY'] = Mistral_TOKEN
llmMist = ChatMistralAI(model="mistral-large-latest", temperature =0)

Function to ingest Appwall security and system DBs in this example and return pandas dataframes

In [ ]:

def epochtotzone(ep,tzo= None):
  """
  Convert epoch values in db to DateTime
  """
  if not tzo:
    tzo = "Asia/Jerusalem"
  tzn=timezone(tzo)
  try:
      t=datetime.datetime.utcfromtimestamp(ep).replace(tzinfo=pytz.utc)
      x= t.astimezone(tzn)
      return t.strftime("%Y-%m-%dT%H:%M:%S")


def aw_to_pd(f,ftype='security',tzone=None):
  """
  Appwall security and system db to pandas dataframe
  """


  jsonlist=[]

  if 'security' in ftype:
      colls= ['DateTime','TargetPort','TargetIP','TransID','TunnelID','IsPassiveMode','Title','URI','Description']
  elif 'system' in ftype:
      colls=['DateTime','Title','Description']

  con = sqlite3.connect(f)
  con.text_factory = lambda b: b.decode(errors = 'ignore') # added to ignore utf-8 decode failure
  cursor = con.cursor()
  cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
  qu='SELECT * from %s'%'Events'
  df=pd.read_sql(qu,con)
  df=df[~df['DateTime'].isna()]
  df['DateTime']=df['DateTime'].apply(epochtotzone,args=(tzone,))
  df['DateTime'] = pd.to_datetime(df['DateTime'])
  df=df[colls]
  df['Date']= df['DateTime']
  df = df.drop('DateTime', axis=1, inplace=False)
  df['Date'] = df['Date'].astype(str)
  if 'security' in ftype:
      df['TransID'] = df['TransID'].astype(str)
  return df

gdrivepath = "/content/drive/MyDrive/Colab Notebooks/My NN stuff/alteon/dbe.security.db"

df_aw = aw_to_pd(gdrivepath)



**Setting up langchain components and pipeline for QA with your db using function calling**

In [ ]:
python_repl_appwall = PythonREPL()
python_repl_appwall.globals['df'] = df_aw


# Permanently changes the pandas settings
python_repl_appwall.run(pd.set_option('display.max_rows', None))
python_repl_appwall.run(pd.set_option('display.max_columns', None))
python_repl_appwall.run(pd.set_option('display.width', None))
python_repl_appwall.run(pd.set_option('display.max_colwidth', 500))


name_appwall = "python_repl_for_pandas_dataframes_appwall_database"

description_appwall_2= """
A Python shell to execute pandas commands on appwall dataframe 'df' which contains appwall database (db).
The column names in this pandas dataframe df can be found in this list = ['Date','TargetPort','TargetIP','TransID','TunnelID','IsPassiveMode','Title','URI','Description']
The dataframe df contains entries in increasing order of time,
such that the latest entry is at the end and earliest entry is at the beginning.
The final answer should always be enclosed in a print() function
"""

repl_tool_appwall = Tool(
    name=name_appwall,
    description=description_appwall_2,
    func=python_repl_appwall.run

)


appwall_llm = llmMist.bind_tools([repl_tool_appwall])

sys_appwall= """
Only if user query is talking about 'appwall' or 'WAF' or 'AW' or 'CWAF' or 'database' or 'db',
do they have questions about the data stored in the pandas dataframe df for appwall database.
"""

prompt_appwall = ChatPromptTemplate.from_messages([
        ("system", sys_appwall),
        ("human", "{query}"),
        ])

setupinputs = {"query": RunnablePassthrough()}


def check_print(result: str) -> str:
  if 'print' in result:
    return result
  else:
    return f"print({result})"

def out_parser_appwall(result: str) -> str:

  """Parses the output of python repl and returns the final formatted output"""

  no_answer = "I couldnt locate what you are asking for in the appwall db."
  attributeerr = "I dont have an answer to your query. Please try to revise your question and ask again. "
  if not result:
    return no_answer
  if 'Empty DataFrame' in result:
    return no_answer
  if 'AttributeError' in result:
    return attributeerr
  return result

def toolingfunc_appwall(x: str) -> str:
  try:
    return x.tool_calls[0]["args"]['__arg1']
  except:
    # No tool/arg found. This need to be different message encouraging user to rephrase the question
    return "print('I dont have an answer to your query. Kindly try rephrasing your question or adding more specificity to it.')"


def notebook_rate_print(data: str) -> str:
  if not data:
    return data
  data_size = len(data)
  if data_size > 999000:
    return "Response data size exceeds display limit and will only be partially displayed. Please refine your query and ask again!\n\n" +data[:999000]
  else:
    return data

appwall_chain = setupinputs| prompt_appwall | appwall_llm | RunnableLambda(toolingfunc_appwall) | RunnableLambda(check_print) | python_repl_appwall.run | RunnableLambda(notebook_rate_print) | RunnableLambda(out_parser_appwall)






**Load up your alteon syslogs and convert to pandas dataframe**

In [ ]:
with open("alteon_syslogs.txt") as fd:
  txtdocuments = fd.read()

txts = txtdocuments.splitlines()
txts = [i[i.find(",2024")+1:] for i in txts]
dii = {"syslogs":txts}
df_syslogs = pd.DataFrame(dii)


##### prepending each entry with Month-day-year-time #################

from datetime import datetime

def prependdt(row):
  # Regular expression pattern to match the datetime format
  pattern = r"\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}[\+\-]\d{2}:\d{2}"
  time_str = re.findall(pattern, row)
  if len(time_str) > 0:
    time_str= time_str[0]
    #time_str = "2024-02-08T15:16:40+05:30"

    # Parse the string with format specifier considering timezone offset
    datetime_obj = datetime.strptime(time_str, "%Y-%m-%dT%H:%M:%S%z")

    # Format the datetime object as desired (Month-day-year-time)
    formatted_time = datetime_obj.strftime("%B-%d-%Y-%H:%M:%S")

    return formatted_time + " " + row
  else:
    print ("emty timestr: ", time_str, row)
    return row

df_syslogs['syslogs'] = df_syslogs['syslogs'].apply(prependdt)


########## convert to string lower for all data in syslog ###########

df_syslogs['syslogs'] = df_syslogs['syslogs'].str.lower()


**Setup syslog langchain components and pipeline**

In [ ]:
python_repl_syslogs = PythonREPL()
python_repl_syslogs.globals['df'] = df_syslogs

# Permanently changes the pandas settings
python_repl_syslogs.run(pd.set_option('display.max_rows', None))
python_repl_syslogs.run(pd.set_option('display.max_columns', None))
python_repl_syslogs.run(pd.set_option('display.width', None))
python_repl_syslogs.run(pd.set_option('display.max_colwidth', 500))

name_syslogs = "python_repl_for_pandas_dataframes_alteon_syslogs"

description_syslogs= """
A Python shell to execute pandas commands on dataframe 'df' which contains alteon syslogs.
There is only one column in this df called 'syslogs'.
The alteon syslogs are stored in df['syslogs'] in increasing order of time,
such that the latest syslog is at the end and earliest syslog is at the beginning.
Output should be only the pandas commands converted to dict using to_dict() and no other words.
The final answer should always be enclosed in a print() function
"""

repl_tool_syslogs = Tool(
    name=name_syslogs,
    description=description_syslogs,
    func=python_repl_syslogs.run

)

syslogs_llm = llmMist.bind_tools([repl_tool_syslogs])

sys_syslogs="""
You are an AI assistant that take user queries about alteon syslogs and events as input and only returns pandas commands to execute on dataframe 'df' which contains alteon syslogs.
There is only one column in this df called 'syslogs'.
The alteon syslogs are stored in df['syslogs'] in increasing order of time,
such that the latest syslog is at the end and earliest syslog is at the beginning.
The pandas commands should be case-insensitive.
Output should be only the pandas commands converted to dict using to_dict() and no other words.
The final answer should always be enclosed in a print() function
"""
prompt_syslogs = ChatPromptTemplate.from_messages([
        ("system", sys_syslogs),
        ("human", "{query}"),
        ])


def check_print(result: str) -> str:
  if 'print' in result:
    return result
  else:
    return f"print({result})"

def out_parser_syslogs(result: str) -> str:

  """Parses the output of python repl and returns the final formatted output"""
  no_answer = "I couldnt locate what you are asking for in the alteon syslogs"

  if not result:
    return no_answer
  if "df['syslogs']" in result:
    return no_answer
  finalls = []
  outtype = None

  if result[0] =="[":
    outtype = "list"
  elif result[0] == "{":
    outtype = "dict"
  else:
    outtype = "string"

  if outtype =="list":

    ls = list(eval(result))
    if not ls:
      return no_answer
    if isinstance(ls[0], dict):
      for i in ls:
        finalls.append("".join(i.values()))
      if finalls:
        return "\n".join(finalls)
      else:
        return no_answer
    else:
      for i in ls:
        finalls.append(i)
      if finalls:
        return "\n".join(finalls)
      else:
        return no_answer

  elif outtype =="dict":

    ls = dict(eval(result))
    if not ls:
      return no_answer
    for i in ls:
      if isinstance(ls[i], dict):
        for j in ls[i]:
          finalls.append(ls[i][j])
        if finalls:
          return "\n".join(finalls)
        else:
          return no_answer
      else:
        for i in ls:
          finalls.append(ls[i])
        if finalls:
          return "\n".join(finalls)
        else:
          return no_answer

  elif outtype == "string":
    return result

def toolingfunc_syslogs(x: str) -> str:
  try:
    return x.tool_calls[0]["args"]['__arg1']
  except:
    # No tool/arg found. This need to be different message encouraging user to rephrase the question
    return "print('I dont have an answer to your query. Kindly try rephrasing your question or adding more specificity to it.')"

def extractcontentonly(x) -> str:
  if not x.content:
    return "print('I couldnt find an answer to your query. Kindly try rephrasing your question or adding more specificity to it.')"
  try:
    return x.content
  except:
    # No tool/arg found. This need to be different message encouraging user to rephrase the question
    return "print('I dont have an answer to your query. Kindly try rephrasing your question or adding more specificity to it.')"

def notebook_rate_print(data: str) -> str:

  if not data:
    return data
  data_size = len(data)
  if data_size > 999000:
    return "Response data size exceeds display limit and will only be partially displayed. Please refine your query and ask again!\n\n" +data[:999000]
  else:
    return data

setupinputs = {"query": RunnablePassthrough()}

syslogs_chain = setupinputs| prompt_syslogs | syslogs_llm | RunnableLambda(toolingfunc_syslogs) | RunnableLambda(check_print) | python_repl_syslogs.run | RunnableLambda(out_parser_syslogs) |RunnableLambda(notebook_rate_print)




**Load up chroma vector db with REST API guide chunk embeddings**

In [ ]:
model_name = "Alibaba-NLP/gte-large-en-v1.5"
model_kwargs = {'device': 'cpu', 'trust_remote_code': True}
encode_kwargs = {'normalize_embeddings': False}

hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

!unzip "/content/drive/MyDrive/Colab Notebooks/My NN stuff/alteon/chroma_db_rest.zip"
persist_directory = 'fulldb'
vectordb_v1 = Chroma(persist_directory=persist_directory, embedding_function=hf)
print(vectordb_v1._collection.count())

In [ ]:
template6 = """Given the context information below and not prior knowledge, answer the question.
The context provides REST and Web API information on the Alteon.
Think step by step before answering the question.
The user is most interested in the URLs found in the context.
From the context provided, extract all relevant URLs and descriptions to those URLs ( if any ), that possibly answer the question.
The final answer should not change or modify the URLs and can contain several URLs.

Context: {context}

Question: {query}

Answer: """

#Check if there is anything extraneous in the final answer that can be removed.

prompt_template = PromptTemplate(
    input_variables=["context","query"],
    template=template6
)


retriever_v1 = vectordb_v1.as_retriever(search_type="similarity", search_kwargs={"k": 10})
compressor = CohereRerank(top_n=7)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever_v1
)


def get_citations(docs):
  cites = []
  for doc in docs:
    cites.append(doc.metadata['source'])
  return "\n\n".join(cites)

def return_only_answer_source_docs(response, citations_only= True):
  cites = []
  docs= response["context"]
  for doc in docs:
    cites.append(doc.metadata['source'])

  idx = response["answer"].find("Answer:")
  if citations_only:
    respstr = response["answer"][idx+7:] + "\n\nSources:\n\n" + "\n".join(cites)
  else:
    respstr = response
  return respstr



rag_chain_from_docs = (
    prompt_template
    | llmMist
    | StrOutputParser()
)


setupandret = RunnableParallel({"context": compression_retriever, "query": RunnablePassthrough()})
rag_chain_with_source = setupandret.assign(answer=rag_chain_from_docs)
RESTapi_chain = rag_chain_with_source | RunnableLambda(return_only_answer_source_docs)


**Keyword router**

In [ ]:
def query_classifier(query):

  q = query.lower()
  cls = {'class':'','query':query}

  if 'rest' in q or 'api call' in q or 'ui' in q or 'gui' in q or 'api' in q or 'url' in q:
    cls['class'] = 'RESTapi'
    return cls

  elif 'appwall' in q or 'cwaf' in q or 'waf' in q or 'database' in q or 'db' in q or 'aw' in q:
    cls['class'] = 'appwall'
    return cls

  elif 'syslog' in q or 'alteon syslog' in q or 'alteon log' in q:
    cls['class'] = 'syslogs'
    return cls

  else:
    cls['class'] = 'Other'
    return cls

def chain_selector(clsdict):

  otherans = """Hello! I am a chat service to talk to Alteon syslogs, or Appwall db. You can also ask me how to make REST calls to the Alteon.
Please phrase your query indicating which service you are interested in and ask again.
For example if you want to talk to Alteon syslogs , you can trying asking:
Are there any cpu events in the alteon syslogs ?
  """

  if clsdict['class'] == 'RESTapi':
    return RESTapi_chain.invoke(clsdict['query'])

  elif clsdict['class'] == 'appwall':
    return appwall_chain.invoke({'query':clsdict['query']})

  elif clsdict['class'] == 'syslogs':
    return syslogs_chain.invoke({'query':clsdict['query'].lower()})

  else:
    return otherans

full_chain = RunnableLambda(query_classifier) | RunnableLambda(chain_selector)

def chained_qa(message, history):
  return full_chain.invoke(message)




**Gradio app for web interface**

In [ ]:

if __name__ == "__main__":
    gr.ChatInterface(
        chained_qa,
        chatbot=gr.Chatbot(height=300),
        textbox=gr.Textbox(placeholder="Talk to Alteon syslogs, WAF events or ask me how to make Alteon REST API calls", container=False, scale=7),
        title="Radware Services",
#        description="Alteon syslogs, WAF events & Alteon REST API",
        theme="soft",
        examples= ["In AW DB how many entries under Description column contain AllowList",
                   "Are there any CPU events in the alteon syslogs ?",
                   "How to check Alteon syslog table via rest call?"],
        cache_examples=False,
        retry_btn=None,
        undo_btn="Delete Previous",
        clear_btn="Clear",
    ).launch(share= True)